# Trials with the Kaiser population

The goal of this notebook is to illustrate how to setup and run trials with the Kaiser population.

In [12]:
import os
microsimDir = "/Users/deligkaris.1/OneDrive - The Ohio State University Wexner Medical Center/MICROSIM/CODE/microsim"
os.chdir(microsimDir)

import pandas as pd

from microsim.person_filter import PersonFilter
from microsim.trials.trial_description import KaiserTrialDescription
from microsim.trials.trial_type import TrialType
from microsim.trials.trial import Trial
from microsim.trials.trial_outcome_assessor_factory import TrialOutcomeAssessorFactory
from microsim.treatment import TreatmentStrategiesType, DefaultTreatmentsType
from microsim.treatment_strategy_repository import TreatmentStrategyRepository
from microsim.bp_treatment_strategies import SprintForSbpOnlyTreatment
from microsim.statin_treatment_strategies import StatinTreatmentStrategy

In [2]:
pd.set_option('future.no_silent_downcasting', True) #make the calculation here future-proof

## A relatively simple Kaiser trial

In [3]:
pf=None #person filter will not be used for inclusion/exclusion, everyone is eligible
sampleSize = 300000
td = KaiserTrialDescription(trialType = TrialType.COMPLETELY_RANDOMIZED,
                            blockFactors=list(),
                            sampleSize = sampleSize, 
                            duration = 4, 
                            treatmentStrategies = "sprint", #a simple way to specify a single treatment strategy
                            nWorkers = 5, 
                            personFilters=pf,
                            wmhSpecific=True) #use WMH-specific risks for the Kaiser outcomes
td

Trial Description
	Trial type: 1
	Block factors: []
	Sample size: 300000
	Duration: 4
	Treatment strategies: ['bp', 'statin']
	Number of workers: 5
	Person filters: 
	 None
	Population type: PopulationType.KAISER

In [4]:
tr = Trial(td)
tr

Trial Description
	Trial type: 1
	Block factors: []
	Sample size: 300000
	Duration: 4
	Treatment strategies: ['bp', 'statin']
	Number of workers: 5
	Person filters: 
	 None
	Population type: PopulationType.KAISER
Trial
	Trial completed: False

In [5]:
tr.print_covariate_distributions()

                          Printing covariate information for people still alive...
                          self=treated, alive people count=  300000               other=control, alive people count=  300000  
                          self=treated, unique alive people count=  193444        other=control, unique alive people count=  193306  
                          self                                                    other
                          -----------------------------------------------------   -----------------------------------------------------
                          min      0.25    med     0.75     max    mean     sd     min     0.25    med     0.75     max    mean     sd
                          -----------------------------------------------------   -----------------------------------------------------
                    age    45.0    59.0    65.0    72.0   110.0    65.8     9.5    45.0    59.0    65.0    72.0   110.0    65.9     9.5
                    sbp  

In [6]:
%%time
tr.run()
tr

/Users/deligkaris.1/Library/Caches/pypoetry/virtualenvs/microsim-54NoInYP-py3.9/lib/python3.9/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


Trial is completed.
CPU times: user 3min 57s, sys: 6min 31s, total: 10min 29s
Wall time: 22min 13s


Trial Description
	Trial type: 1
	Block factors: []
	Sample size: 300000
	Duration: 4
	Treatment strategies: ['bp', 'statin']
	Number of workers: 5
	Person filters: 
	 None
	Population type: PopulationType.KAISER
Trial
	Trial completed: True

In [7]:
toa = TrialOutcomeAssessorFactory.get_trial_outcome_assessor()
toa

Trial Outcome Assessor
	Assessments:
		Name: death                    Analysis: logistic       
		Name: anyEvent                 Analysis: logistic       
		Name: vascularEventOrDeath     Analysis: logistic       
		Name: vascularEvent            Analysis: logistic       
		Name: qalys                    Analysis: linear         
		Name: meanGCP                  Analysis: linear         
		Name: lastGCP                  Analysis: linear         
		Name: cogEvent                 Analysis: logistic       
		Name: deathCox                 Analysis: cox            
		Name: cogEventCox              Analysis: cox            
		Name: vascularEventOrDeathCox  Analysis: cox            
		Name: strokeRR                 Analysis: relativeRisk   
		Name: miRR                     Analysis: relativeRisk   
		Name: cvRR                     Analysis: relativeRisk   
		Name: dementiaRR               Analysis: relativeRisk   
		Name: ciRR                     Analysis: relativeRisk   
		Name: dementiaOrC

In [8]:
tr.analyze(toa)
tr

Trial Description
	Trial type: 1
	Block factors: []
	Sample size: 300000
	Duration: 4
	Treatment strategies: ['bp', 'statin']
	Number of workers: 5
	Person filters: 
	 None
	Population type: PopulationType.KAISER
Trial
	Trial completed: True
Trial results:
	Analysis: linear
	                                         Z      Intercept           Z SE         pValue
	                    qalys:            0.00           3.62           0.00           0.87
	                  meanGCP:            0.05          52.07           0.02           0.01
	                  lastGCP:            0.09          50.46           0.02           0.00
	Analysis: logistic
	                                         Z      Intercept           Z SE         pValue
	                    death:           -0.00          -2.16           0.01           0.56
	                 anyEvent:           -0.01           0.87           0.01           0.05
	     vascularEventOrDeath:           -0.00          -1.61           0.01         

In [9]:
tr.print_treatment_strategy_variables_distribution()

                          self=treated, alive people count=  269142  
                          self=treated & alive, unique people count=  180372  
                          self
                          -----------------------------------------------------
                          proportions
                          -----------
            bpMedsAdded
                      0   0.93
                      1   0.04
                      2   0.02
                      3   0.01
                      4   0.00


## A more complex Kaiser trial

We will create a person filter with user-specified criteria.

In [10]:
pf = PersonFilter() 
#pf.add_filter("person", 
#              "lowCvLimit", 
#              lambda x: (CVModelRepository().select_outcome_model_for_person(x).get_risk_for_person(x, years=10)<0.075))
pf.add_filter("df",
              "noStatin",
              lambda x: x[DefaultTreatmentsType.STATIN.value]==0)

We will initialize a treatment strategy that includes a user-specified CV risk cutoff for statins added and a blood pressure treatment.

In [13]:
ts = TreatmentStrategyRepository()
ts._repository[TreatmentStrategiesType.BP.value] = SprintForSbpOnlyTreatment()
ts._repository[TreatmentStrategiesType.STATIN.value] = StatinTreatmentStrategy(cvRiskCutoff=0.075)

Setup the trial description and then follow the usual steps.

In [14]:
td = KaiserTrialDescription(trialType = TrialType.COMPLETELY_RANDOMIZED,
                            blockFactors=list(),
                            sampleSize = sampleSize, 
                            duration = 4, 
                            treatmentStrategies = ts, #a more complex way to specify treatment strategies
                            nWorkers = 5, 
                            personFilters=pf,
                            wmhSpecific=True) #use WMH-specific risks for the Kaiser outcomes
td

Trial Description
	Trial type: 1
	Block factors: []
	Sample size: 300000
	Duration: 4
	Treatment strategies: ['bp', 'statin']
	Number of workers: 5
	Person filters: 
	 Person Filters:
	    filter type   filter name    
	             df   noStatin       

	Population type: PopulationType.KAISER

In [15]:
tr = Trial(td)
tr

Trial Description
	Trial type: 1
	Block factors: []
	Sample size: 300000
	Duration: 4
	Treatment strategies: ['bp', 'statin']
	Number of workers: 5
	Person filters: 
	 Person Filters:
	    filter type   filter name    
	             df   noStatin       

	Population type: PopulationType.KAISER
Trial
	Trial completed: False

In [16]:
tr.print_covariate_distributions()

                          Printing covariate information for people still alive...
                          self=treated, alive people count=  300000               other=control, alive people count=  300000  
                          self=treated, unique alive people count=  143424        other=control, unique alive people count=  143133  
                          self                                                    other
                          -----------------------------------------------------   -----------------------------------------------------
                          min      0.25    med     0.75     max    mean     sd     min     0.25    med     0.75     max    mean     sd
                          -----------------------------------------------------   -----------------------------------------------------
                    age    45.0    57.0    63.0    70.0   111.0    63.9     9.3    45.0    57.0    63.0    70.0   110.0    63.9     9.3
                    sbp  

In [17]:
%%time
tr.run()
tr

/Users/deligkaris.1/Library/Caches/pypoetry/virtualenvs/microsim-54NoInYP-py3.9/lib/python3.9/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


Trial is completed.
CPU times: user 5min 3s, sys: 18min 41s, total: 23min 44s
Wall time: 47min 23s


Trial Description
	Trial type: 1
	Block factors: []
	Sample size: 300000
	Duration: 4
	Treatment strategies: ['bp', 'statin']
	Number of workers: 5
	Person filters: 
	 Person Filters:
	    filter type   filter name    
	             df   noStatin       

	Population type: PopulationType.KAISER
Trial
	Trial completed: True

In [18]:
toa = TrialOutcomeAssessorFactory.get_trial_outcome_assessor()
toa

Trial Outcome Assessor
	Assessments:
		Name: death                    Analysis: logistic       
		Name: anyEvent                 Analysis: logistic       
		Name: vascularEventOrDeath     Analysis: logistic       
		Name: vascularEvent            Analysis: logistic       
		Name: qalys                    Analysis: linear         
		Name: meanGCP                  Analysis: linear         
		Name: lastGCP                  Analysis: linear         
		Name: cogEvent                 Analysis: logistic       
		Name: deathCox                 Analysis: cox            
		Name: cogEventCox              Analysis: cox            
		Name: vascularEventOrDeathCox  Analysis: cox            
		Name: strokeRR                 Analysis: relativeRisk   
		Name: miRR                     Analysis: relativeRisk   
		Name: cvRR                     Analysis: relativeRisk   
		Name: dementiaRR               Analysis: relativeRisk   
		Name: ciRR                     Analysis: relativeRisk   
		Name: dementiaOrC

In [19]:
tr.analyze(toa)
tr

Trial Description
	Trial type: 1
	Block factors: []
	Sample size: 300000
	Duration: 4
	Treatment strategies: ['bp', 'statin']
	Number of workers: 5
	Person filters: 
	 Person Filters:
	    filter type   filter name    
	             df   noStatin       

	Population type: PopulationType.KAISER
Trial
	Trial completed: True
Trial results:
	Analysis: linear
	                                         Z      Intercept           Z SE         pValue
	                    qalys:            0.01           3.70           0.00           0.00
	                  meanGCP:            0.04          52.74           0.02           0.02
	                  lastGCP:            0.06          51.13           0.02           0.01
	Analysis: logistic
	                                         Z      Intercept           Z SE         pValue
	                    death:           -0.03          -2.41           0.01           0.00
	                 anyEvent:           -0.02           0.80           0.01           0.00


In [20]:
tr.print_treatment_strategy_variables_distribution()

                          self=treated, alive people count=  276042  
                          self=treated & alive, unique people count=  138272  
                          self
                          -----------------------------------------------------
                          proportions
                          -----------
            bpMedsAdded
                      0   1.00
           statinsAdded
                      0   0.34
                      1   0.66
